In [7]:
import numpy as np
from layers import *
from collections import OrderedDict
from loss import *
from activation import *

In [23]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size):
        self.W = {}
        self.W['W1'] = np.random.randn(input_size,hidden_size)
        self.W['b1'] = np.random.randn(hidden_size)
        self.W['W2'] = np.random.randn(hidden_size, output_size)
        self.W['b2'] = np.random.randn(output_size)
        
        # self.acc_val = []
        self.loss_val = []
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.W['W1'],self.W['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.W['W2'],self.W['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        loss = self.lastLayer.forward(y,t)
        return loss

    def numerical_gradient(self,x,t):
        f = lambda W : self.loss(x,t)
        grads = {}
        grads['W1'] = numerical_gradient(f, self.W['W1']) # 밖에서 만든 numerical_gradient함수
        grads['b1'] = numerical_gradient(f, self.W['b1'])
        grads['W2'] = numerical_gradient(f, self.W['W2'])
        grads['b2'] = numerical_gradient(f, self.W['b2'])
        return grads
    
    # def gradient_descent(self,epochs):
    
    def gradient(self,x,t):
        self.loss(x,t)
        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dw
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dw
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
        
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        acc = sum(y == t)/x.shape[0]
        return acc
    
    def fit(self,x,t,epochs=1000,lr=1e-3,verbos=1):
        for epoch in range(epochs):
            grads = self.gradient(x,t)
            for key in grads.keys():
                self.W[key] -= lr*grads[key]
            
            # self.acc_val.append(self.accuracy(x,t))
            self.loss_val.append(self.loss(x,t))
            if verbos == 1:
                print(f'epoch:{epoch}, acc:{self.accuracy(x,t)}, loss:{self.loss(x,t)}')

In [9]:
x = np.random.randn(10,5)
t = np.random.randn(10,4)
input_size = x.shape[1]
hidden_size = 3
output_size = 4

W = {}
W['W1'] = np.random.randn(input_size,hidden_size)
W['b1'] = np.random.randn(hidden_size)
W['W2'] = np.random.randn(hidden_size, output_size)
W['b2'] = np.random.randn(output_size)

layers = OrderedDict()
layers['Affine1'] = Affine(W['W1'],W['b1'])
layers['Relu1'] = Relu()
layers['Affine2'] = Affine(W['W2'],W['b2'])
layers

OrderedDict([('Affine1', <layers.Affine at 0x7f5474e4d640>),
             ('Relu1', <layers.Relu at 0x7f5474e4d940>),
             ('Affine2', <layers.Affine at 0x7f5474e4dbe0>)])

In [21]:
layers = list(())
layers.reverse()
layers
layers.values

AttributeError: 'list' object has no attribute 'values'

In [10]:
for layer in layers.values():
    x = layer.forward(x)

In [11]:
y = softmax(x)
loss = cross_entropy_error(y,t)

In [13]:
dout = 1
dout = dout*(y - t)/y.shape[0]

In [15]:
dout.shape

(10, 4)

In [ ]:
def predict(self, x):
    w1,w2 = self.W['W1'], self.W['W2']
    b1,b2 = self.W['b1'], self.W['b2']
    a1 = np.dot(x,W1) + b1
    z1 = relu(a1)
    a2 = np.dot(z1,W2) + b2
    y = softmax(a2)